_Alex Malz (NYU) & David Mykytyn (NYU)_

In [ ]:
import daft

# pip install jupyter_contrib_nbextensions
# %%javascript
# MathJax.Hub.Config({
#     TeX: { equationNumbers: { autoNumber: "AMS" } }
# });

A first stab at a PGM for the HR diagram (parameterized by $HR$) of open clusters $k$ (parameterized by $OC$) with double/triple stars -- groups $\{j'\}$ of physical stars $j$ (with true latent colors $c$, magnitudes $M$, kinematics $v$, and physical locations $x$) identified as a single star $i$ with with observed properties (color $\hat{c}$, magnitude $\hat{M}$, parallax $\hat{\varpi}$, proper motion $\hat{v}_{\perp}$, and angular position $\hat{x}_{\perp}$) that are influenced by the stars $\{j'\}$ that went into it.
So far, we're assuming that star clusters are independent, i.e. that we do not have any interlopers from other stellar populations.

In [ ]:
#initialize the PGM
pgm = daft.PGM([5., 6.5], origin=[0, 0])

#undesired hyperparameters
pgm.add_node(daft.Node("physics", r"$\vec{\theta}$", 2.5, 6.))

#desired hyperparameters
pgm.add_node(daft.Node("HR", r"HR", 2., 5.))
pgm.add_node(daft.Node("OC", r"OC", 3., 5.))

#latent variables/parameters
pgm.add_node(daft.Node("photometry", r"$M, c$", 2., 3.5))
pgm.add_node(daft.Node("astrometry", r"$v, x$", 3., 4.))
pgm.add_node(daft.Node("multiplicity", r"$\{j'\}$", 2.5, 2.25))

#known constant parameters
pgm.add_node(daft.Node("resolution", r"$\delta_{x}$", 4.5, 3., fixed=True))
pgm.add_node(daft.Node("mag limits", r"$\delta_{M}$", 0.5, 1., fixed=True))
pgm.add_node(daft.Node("limits", r"$\delta_{\varpi}, \delta_{v}$", 4.5, 1., fixed=True))

#data
pgm.add_node(daft.Node("phot observations", r"$\hat{M}, \hat{c}$", 2., 1.5, aspect=1., observed=True))
pgm.add_node(daft.Node("ast observations", r"$\hat{\varpi}, \hat{v}_{\perp}, \hat{x}_{\perp}$", 3., 1.5, aspect=1.75, observed=True))


# Add in the edges.
# pgm.add_edge("", "")
pgm.add_edge("physics", "HR")
pgm.add_edge("physics", "OC")

pgm.add_edge("HR", "photometry")
pgm.add_edge("OC", "astrometry")
pgm.add_edge("astrometry", "multiplicity")

pgm.add_edge("resolution", "multiplicity")
pgm.add_edge("resolution", "ast observations")
pgm.add_edge("limits", "ast observations")
pgm.add_edge("mag limits", "phot observations")
pgm.add_edge("photometry", "phot observations")
pgm.add_edge("astrometry", "ast observations")
pgm.add_edge("multiplicity", "phot observations")
pgm.add_edge("multiplicity", "ast observations")

# plates
pgm.add_plate(daft.Plate([1.5, 3., 2., 1.5], label=r"physical stars $j$", bbox={"color": "none"}))
pgm.add_plate(daft.Plate([1., 0.5, 3., 5.], label=r"star clusters $k$", bbox={"color": "none"}))
pgm.add_plate(daft.Plate([1.5, 1., 2., 1.75], label=r"observed stars $i$", bbox={"color": "none"}))

# Render and save.
pgm.render()
pgm.figure.savefig("OCmodel.png", dpi=250)

Our immediate goal is a forward model that produces $p(HR \mid \{\hat{M}, \hat{c}\})$, where $HR$ parameterizes the space of $\{M, c\}$, i.e. $p(M, c \mid HR) = f(HR)$, by marginalizing over $s/b/t$ and $M, c$.
(A reach goal is to also get $p(HR \mid \vec{\theta})$.)
So far, we're thinking that $f(HR)$ should be a Gaussian process.

The forward model is as follows:
\begin{eqnarray}
\{M, c\}_{j} & \sim & p(M, c \mid HR_{k})\\
\{v, x\}_{j} & \sim & p(v, x \mid OC_{k})\\
\{j'\}_{i} & \sim & p(\{j\} \mid \{x\}_{j}, \delta_{x})\\
\{\hat{\varpi}, \hat{v}_{\perp}, \hat{x}_{\perp}\}_{i} & \sim & p(\hat{\varpi}, \hat{v}_{\perp}, \hat{x}_{\perp} \mid \{v, x\}_{j'}, \delta_{\varpi}, \delta_{v}, \delta_{x})\\
\{\hat{M}, \hat{c}\}_{i} & \sim & p(\hat{M}, \hat{c} \mid \{M, c\}_{j'}, \delta_{M})
\end{eqnarray}

Knowing the definition of magnitude in terms of flux and assuming that the observed flux of "star" $i$ is the sum of the fluxes of the physical stars $\{j'\}$ gives us $p(\hat{M}, \hat{c} \mid \{M, c\}_{j'}, \delta_{M})$, plus the Gaussian error due to the magnitude limit $\delta_{M}$.
We're not really sure why [Widmark+18](https://arxiv.org/pdf/1801.08547) uses a sum of Gaussians for this.

For now, let's assume that $p(\hat{\varpi}, \hat{v}_{\perp}, \hat{x}_{\perp} \mid \{v, x\}_{j'}, \delta_{\varpi}, \delta_{v}, \delta_{x})$ is a Gaussian around the true values derived from the definition of $\varpi$ and the trigonometry for the projected observations.
Note that [Gagne+18](https://arxiv.org/pdf/1801.09051.pdf) carefully outlines the inverse of the math we seek for a forward model.

Also, knowing the definitions of the resolution limit, we can construct $p(\{j\} \mid \{x\}_{j}, \delta_{x})$ as a delta function kind of thing, so it's 1 for $\{j\} = j', j'', \dots$ if $|x_{j'} - x_{j''}| < \delta_{x}$ and 0 otherwise, to consider groups of $\{j'\}$ that are perceived as $i$; ultimately, this would also have a Gaussian component but I'm just starting with an oversimplification.

The only other things we need are a 2D Gaussian Process we believe for $p(M, c \mid HR_{k})$ and some model for $p(v, x \mid OC_{k})$, presumably derived from physics.
It looks like [Gagne+18](https://arxiv.org/pdf/1801.09051.pdf) also gives a physically motivated $p(v, x \mid OC_{k})$.

What we really want to do, though, is posterior inference, so we need to turn this around and/or figure out what parts of this to forward model with sampling and what parts to analytically invert, etc.!